In [1]:
from gurobipy import *

In [2]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal
class BB:
    def __init__(self,ship_num,stack_num,height,n_init,size,size_,beta,mean,cov,OC,zk,aaa):
        self.ship_num=ship_num
        self.stack_num=stack_num
        self.height=height
        self.n_init=n_init
        self.beta=beta
        self.size=size
        self.size_=size_
        self.mean=mean
        self.cov=cov
        self.OC=OC
        self.zk=zk
        self.O1=[i+1 for i in range(self.ship_num)]
        self.S=[i+1 for i in range(self.stack_num)]
        self.H=[i+1 for i in range(self.height)]
        self.P=[i+1 for i in range(self.ship_num)]
        self.f=stack_num*height-sum(self.n_init)
        self.aaa=aaa

        # 期待値と分散共分散行列の準備
        data_1 = np.random.multivariate_normal(mean, cov, size=self.size)

        O_=np.argsort(data_1)
        O_=O_+np.ones((size,ship_num)).astype(int)
        global O
        self.O=O_.tolist()
        O=self.O

    def get_penalty(self,result):
        self.result=result
        self.penalty=[]
        for k in self.O:
            OO=k
            a=0
            for j in range(self.stack_num):
                for i in range(1,self.height):
                    for i_ in range(i+1,self.height+1):
                        if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
                            if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
                                # print(j+1,height-i+1,O)
                                a+=1
                                # print("penalty!")
                                    # print(i,i_,j+1,O)
                                break
            self.penalty.append(a)
        self.zk=[i for i, x in enumerate(self.penalty) if x-self.alpha_tmp>0]
        
        for zk_ in self.zk:
            if not zk_ in self.aaa:
                self.aaa.append(zk_)
                self.OC.append(O[zk_])
        return (self.zk,self.OC,self.penalty)

    def solve(self):
        def add_cutting_plane(model,where):
            if where == GRB.callback.MIPSOL:
                c_var,x_var,x_tmp,d_var={},{},{},{}
                for var in self.model._vars:
                    if "u" in var.VarName:
                        u_var=var
                    if "alpha" in var.VarName:
                        alpha_var=var
                        self.alpha_tmp=model.cbGetSolution(var)
                    if "c_" in var.VarName:
                        s=int(var.VarName.split("_")[-3])
                        h=int(var.VarName.split("_")[-2])
                        i=int(var.VarName.split("_")[-1])
                        c_var[s,h,i]=var
                    if "d_" in var.VarName:
                        i=int(var.VarName.split("_")[-1])
                        d_var[i]=var
                    if "x_" in var.VarName:
                        s=int(var.VarName.split("_")[-3])
                        h=int(var.VarName.split("_")[-2])
                        p=int(var.VarName.split("_")[-1])
                        x_var[s,h,p]=var
                        x_tmp[s,h,p]=model.cbGetSolution(var)
                global result
                result=self.get_result(x_tmp)
                print(result)
                zk,OC,penalty=self.get_penalty(result)
                a=[]
                dic={}
                for i in OC:
                    # print(i)
                    if not i in a:
                        a.append(i)
                for j in a:
                    dic[tuple(j)]=OC.count(j)
                print("Len_dic=",len(dic))
                print(len(zk),len(OC))
                for s in self.S:
                    for h in range(2,len(self.H)+1):
                        for h_ in range(1,h):
                            for i,o in enumerate(dic):
                                for j,p in enumerate(o):
                                    model.cbLazy(c_var[s,h,i]>=dic[o]*(quicksum(x_var[s,h,k] for k in o[j:])-quicksum(x_var[s,h_,k] for k in o[j:])))
                for i in range(len(dic)):
                    model.cbLazy(d[i]>=quicksum(c_var[s,h,i] for s in self.S for h in self.H if h!=1)-alpha_var)
                model.cbLazy(u_var>=alpha_var+quicksum(d[i] for i in range(len(dic)))/((1-self.beta)*len(O)))
        self.model=Model("CuttingPlaneCVaRMinimization")
        a=[]
        dic={}
        for i in self.O:
            # print(i)
            if not i in a:
                a.append(i)
        for j in a:
            dic[tuple(j)]=self.O.count(j)
        x,c,d={},{},{}
        for s in self.S:
            for h in self.H:
                for p in self.P:
                    x[s,h,p]=self.model.addVar(vtype="B",name="x_"+str(s)+"_"+str(h)+"_"+str(p))

        for s in self.S:
            for h in range(2,len(self.H)+1):
                for i,o in enumerate(self.O):
                    c[s,h,i]=self.model.addVar(vtype="C",lb=0,name="c_"+str(s)+"_"+str(h)+"_"+str(i))

        for i in range(len(self.O)):
            d[i]=self.model.addVar(vtype="C",lb=0)
        alpha=self.model.addVar(vtype="C",lb=0,name="alpha")
        u=self.model.addVar(vtype="C",lb=0,name="u")

        self.model.update()
        self.model._vars=self.model.getVars()

        for p in self.P:
            self.model.addConstr(quicksum(x[s,h,p] for s in self.S for h in self.H)==self.n_init[p-1])

        for s in self.S:
            for h in self.H:
                self.model.addConstr(quicksum(x[s,h,p] for p in self.P)<=1)

        for s in self.S:
            for h in range(1,len(self.H)):
                self.model.addConstr(quicksum(x[s,h+1,p] for p in self.P)<=quicksum(x[s,h,p] for p in self.P))

        for s in self.S:
            for h in range(2,len(self.H)+1):
                for h_ in range(1,h):
                    for i,o in enumerate(dic):
                        for j,p in enumerate(o):
                            self.model.addConstr(c[s,h,i]>=dic[o]*(quicksum(x[s,h,k] for k in o[j:])-quicksum(x[s,h_,k] for k in o[j:])))
        for i in range(len(dic)):
            self.model.addConstr(d[i]>=quicksum(c[s,h,i] for s in self.S for h in self.H if h!=1)-alpha)
        self.model.addConstr(u>=alpha+quicksum(d[i] for i in range(len(dic)))/((1-self.beta)*len(O)))
        # if len(self.OC)==0:
        #     self.OC=[self.O1]
        # else:
        #     if self.O1 in self.OC:
        #         self.OC=self.OC.remove(self.O1)
        if len(self.OC)>self.size:
            self.OC=self.OC[self.size:]
        print("OC=",self.OC)
        for s in self.S:
            for h in range(2,len(self.H)+1):
                for i,o in enumerate([self.O1]):
                    for j,p in enumerate(o):
                        self.model.addConstr(quicksum(x[s,h,k] for k in o[j:])<=quicksum(x[s,h-1,k] for k in o[j:]))

        for i in range(len([self.O1])):
            self.model.addConstr(d[i]>=quicksum(c[s,h,i] for s in self.S for h in self.H if h!=1)-alpha)

        self.model.addConstr(u>=alpha+quicksum(d[i] for i in range(len(self.O)))/((1-self.beta)*len(self.O)))
        self.model.setObjective(u)
        self.model._vars=self.model.getVars()
        self.model.params.LazyConstraints = 1
        self.model.params.TimeLimit = 3600
        self.model.params.MIPFocus=2
        if self.f>=self.height:
            self.model.optimize()
        else:
            print("実行不可")

    def get_optimal_val(self):
        if self.model.Status==GRB.OPTIMAL:
            self.LB=self.model.ObjVal
            return self.model.ObjVal
        else:
            return None

    def get_optimal_sol(self):
        if self.model.Status==GRB.OPTIMAL:
            x_opt={}
            c_opt={}
            for var in self.model._vars:
                if "x_" in var.VarName:
                    # print(var)
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    p=int(var.VarName.split("_")[-1])
                    x_opt[s,h,p]=var.X
                if "c_" in var.VarName:
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    i=int(var.VarName.split("_")[-1])
                    c_opt[s,h,i]=var.X
                if "alpha" in var.VarName:
                    alpha_opt=var.X
                if "u" in var.VarName:
                    u_opt=var.X
            return (x_opt,c_opt,alpha_opt,u_opt)
        else:
            return None

    def get_result(self,x_tmp):
        self.x_tmp=x_tmp
        EPS=1.e-6
        self.result=np.zeros((self.height,self.stack_num))
        for (s,h,p) in self.x_tmp:
            if self.x_tmp[s,h,p]>EPS:
                self.result[self.height-h][s-1]=int(p)
        self.result=self.result.astype(int)
        return self.result
    
    # def get_penalty(self,result):
    #     if self.model.Status==GRB.OPTIMAL:
    #         self.result=result
    #         self.penalty=[]
    #         x,c,alpha,u=self.get_optimal_sol()
    #         for k in self.O:
    #             OO=k
    #             a=0
    #             for j in range(self.stack_num):
    #                 for i in range(1,self.height):
    #                     for i_ in range(i+1,self.height+1):
    #                         if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
    #                             if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
    #                                 # print(j+1,height-i+1,O)
    #                                 a+=1
    #                                 # print("penalty!")
    #                                     # print(i,i_,j+1,O)
    #                                 break
    #             self.penalty.append(a)
    #         self.zk=[i for i, x in enumerate(self.penalty) if x-alpha>0]
    #         for zk_ in self.zk:
    #             self.OC.append(self.O[zk_])
    #         return (self.zk,self.OC,self.penalty)

In [9]:
# 13m6.5s
size=1000
size_=10000
beta=0.75
inst=[5,4,[2,2,2,2,2,2,2]]
pcvar=[]
import numpy as np
# mean=np.arange(1,ship_num+1)
mean=np.array([1,1.5,4,4.5,5,5.6,6])
covl=[1,1,1.5,1,1,1.5,1,1,1.5]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov=np.zeros((len(inst[2]),len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i==j:
            cov[i][i]=covl[1]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i!=j:
            cov[i][j]=0

OC=[]
EPS=0.1
a=0
UB=1000
LB=0
UB_k=[]
zk=[]
aaa=[]

model=BB(len(inst[2]),inst[0],inst[1],inst[2],size,size_,0.75,mean,cov,OC,zk,aaa)
model.solve()

OC= []
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 44671 rows, 16142 columns and 404939 nonzeros
Model fingerprint: 0x9a95acd1
Variable types: 16002 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3247 rows and 10 columns (presolve time = 5s) ...
Presolve removed 3247 rows and 10 columns
Presolve time: 7.91s
Presolved: 41424 rows, 16132 columns, 368899 nonzeros
Variable types: 16002 continuous, 130 integer (130 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Root relaxation presolved: 41424 rows, 16132 columns, 368899 nonzeros


Ro

In [10]:
a,b,c,d=model.get_optimal_sol()
result=model.get_result(a)
print(result)

[[2 0 0 0 0]
 [2 0 1 1 0]
 [7 3 6 5 4]
 [7 3 6 5 4]]


In [5]:
# 1m22.8s
size=1000
size_=10000
beta=0.75
inst=[4,4,[2,2,2,2,2,2]]
pcvar=[]
import numpy as np
# mean=np.arange(1,ship_num+1)
mean=np.arange(1,len(inst[2])+1)
covl=[2 for i in range(len(inst[2]))]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov=np.zeros((len(inst[2]),len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i==j:
            cov[i][i]=covl[1]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i!=j:
            cov[i][j]=0

OC=[]
EPS=0.1
a=0
UB=1000
LB=0
UB_k=[]
zk=[]
aaa=[]

model=BB(len(inst[2]),inst[0],inst[1],inst[2],size,size_,0.75,mean,cov,OC,zk,aaa)
model.solve()

OC= []
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 3660
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22874 rows, 13098 columns and 185077 nonzeros
Model fingerprint: 0x676aa143
Variable types: 13002 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [4e-03, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 2202 rows and 8 columns
Presolve time: 1.08s
Presolved: 20672 rows, 13090 columns, 160504 nonzeros
Variable types: 13002 continuous, 88 integer (88 binary)
Found heuristic solution: objective 1.9120000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Use crossover to convert LP symmetric solution to basic solution...
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective 0.000

In [5]:
size=1000
size_=10000
beta=0.75
inst=[4,4,[3,3,1,1,2,2]]
pcvar=[]
import numpy as np
mean=np.arange(1,len(inst[2])+1)
covl=[1 for i in range(len(mean))]
# mean=np.array([1,1.5,4,4.5,5,5.6])
# covl=[1,1,1.5,1,1,1.5]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov=np.zeros((len(inst[2]),len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i==j:
            cov[i][i]=covl[1]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i!=j:
            cov[i][j]=0


OC=[]
EPS=0.1
a=0
UB=1000
LB=0
UB_k=[]
zk=[]
aaa=[]

model=BB(len(inst[2]),inst[0],inst[1],inst[2],size,size_,0.75,mean,cov,OC,zk,aaa)
model.solve()

OC= []
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11854 rows, 13098 columns and 96385 nonzeros
Model fingerprint: 0x80f61e75
Variable types: 13002 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [4e-03, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 1710 rows and 8 columns
Presolve time: 0.50s
Presolved: 10144 rows, 13090 columns, 80992 nonzeros
Variable types: 13002 continuous, 88 integer (88 binary)
Found heuristic solution: objective 2.3360000
Root relaxation presolved: 10144 rows, 13090 columns, 80992 nonzeros


Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 1.000000e-02, 361 iterations, 0.17 seconds (0.10 w

In [2]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal
class CuttingPlaneAlgorithm:
    def __init__(self,ship_num,stack_num,height,n_init,size,size_,beta,mean,cov,OC,zk,aaa):
        self.ship_num=ship_num
        self.stack_num=stack_num
        self.height=height
        self.n_init=n_init
        self.beta=beta
        self.size=size
        self.size_=size_
        self.mean=mean
        self.cov=cov
        self.OC=OC
        self.zk=zk
        self.O1=[i+1 for i in range(self.ship_num)]
        self.S=[i+1 for i in range(self.stack_num)]
        self.H=[i+1 for i in range(self.height)]
        self.P=[i+1 for i in range(self.ship_num)]
        self.f=stack_num*height-sum(self.n_init)
        self.aaa=aaa

        # 期待値と分散共分散行列の準備
        data_1 = np.random.multivariate_normal(mean, cov, size=self.size)

        O_=np.argsort(data_1)
        O_=O_+np.ones((size,ship_num)).astype(int)
        global O
        self.O=O_.tolist()
        O=self.O

    def get_penalty(self,result):
        self.result=result
        self.penalty=[]
        for k in self.O:
            OO=k
            a=0
            for j in range(self.stack_num):
                for i in range(1,self.height):
                    for i_ in range(i+1,self.height+1):
                        if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
                            if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
                                # print(j+1,height-i+1,O)
                                a+=1
                                # print("penalty!")
                                    # print(i,i_,j+1,O)
                                break
            self.penalty.append(a)
        self.zk=[i for i, x in enumerate(self.penalty) if x-self.alpha_tmp>0]
        
        for zk_ in self.zk:
            if not zk_ in self.aaa:
                self.aaa.append(zk_)
                self.OC.append(O[zk_])
        return (self.zk,self.OC,self.penalty)

    def solve(self):
        def add_cutting_plane(model,where):
            if where == GRB.callback.MIPSOL:
                c_var,x_var,x_tmp,d_var={},{},{},{}
                for var in self.model._vars:
                    if "u" in var.VarName:
                        u_var=var
                    if "alpha" in var.VarName:
                        alpha_var=var
                        self.alpha_tmp=model.cbGetSolution(var)
                    if "c_" in var.VarName:
                        s=int(var.VarName.split("_")[-3])
                        h=int(var.VarName.split("_")[-2])
                        i=int(var.VarName.split("_")[-1])
                        c_var[s,h,i]=var
                    if "d_" in var.VarName:
                        i=int(var.VarName.split("_")[-1])
                        d_var[i]=var
                    if "x_" in var.VarName:
                        s=int(var.VarName.split("_")[-3])
                        h=int(var.VarName.split("_")[-2])
                        p=int(var.VarName.split("_")[-1])
                        x_var[s,h,p]=var
                        x_tmp[s,h,p]=model.cbGetSolution(var)
                global result
                result=self.get_result(x_tmp)
                print(result)
                zk,OC,penalty=self.get_penalty(result)
                a=[]
                dic={}
                for i in OC:
                    # print(i)
                    if not i in a:
                        a.append(i)
                for j in a:
                    dic[tuple(j)]=OC.count(j)
                print("Len_dic=",len(dic))
                print(len(zk),len(OC))
                for s in self.S:
                    for h in range(2,len(self.H)+1):
                        for h_ in range(1,h):
                            for i,o in enumerate(dic):
                                for j,p in enumerate(o):
                                    model.cbLazy(c_var[s,h,i]>=dic[o]*(quicksum(x_var[s,h,k] for k in o[j:])-quicksum(x_var[s,h_,k] for k in o[j:])))
                for i in range(len(dic)):
                    model.cbLazy(d[i]>=quicksum(c_var[s,h,i] for s in self.S for h in self.H if h!=1)-alpha_var)
                model.cbLazy(u_var>=alpha_var+quicksum(d[i] for i in range(len(dic)))/((1-self.beta)*len(O)))
        self.model=Model("CuttingPlaneCVaRMinimization")
        x,c,d={},{},{}
        for s in self.S:
            for h in self.H:
                for p in self.P:
                    x[s,h,p]=self.model.addVar(vtype="B",name="x_"+str(s)+"_"+str(h)+"_"+str(p))

        for s in self.S:
            for h in range(2,len(self.H)+1):
                for i,o in enumerate(self.O):
                    c[s,h,i]=self.model.addVar(vtype="c",lb=0,name="c_"+str(s)+"_"+str(h)+"_"+str(i))

        for i in range(len(self.O)):
            d[i]=self.model.addVar(vtype="C",lb=0)
        alpha=self.model.addVar(vtype="C",lb=0,name="alpha")
        u=self.model.addVar(vtype="C",lb=0,name="u")

        self.model.update()
        self.model._vars=self.model.getVars()

        for p in self.P:
            self.model.addConstr(quicksum(x[s,h,p] for s in self.S for h in self.H)==self.n_init[p-1])

        for s in self.S:
            for h in self.H:
                self.model.addConstr(quicksum(x[s,h,p] for p in self.P)<=1)

        for s in self.S:
            for h in range(1,len(self.H)):
                self.model.addConstr(quicksum(x[s,h+1,p] for p in self.P)<=quicksum(x[s,h,p] for p in self.P))
        # if len(self.OC)==0:
        #     self.OC=[self.O1]
        # else:
        #     if self.O1 in self.OC:
        #         self.OC=self.OC.remove(self.O1)
        if len(self.OC)>self.size:
            self.OC=self.OC[self.size:]
        print("OC=",self.OC)
        for s in self.S:
            for h in range(2,len(self.H)+1):
                for i,o in enumerate([self.O1]):
                    for j,p in enumerate(o):
                        self.model.addConstr(quicksum(x[s,h,k] for k in o[j:])<=quicksum(x[s,h-1,k] for k in o[j:]))

        for i in range(len([self.O1])):
            self.model.addConstr(d[i]>=quicksum(c[s,h,i] for s in self.S for h in self.H if h!=1)-alpha)

        self.model.addConstr(u>=alpha+quicksum(d[i] for i in range(len(self.O)))/((1-self.beta)*len(self.O)))
        self.model.setObjective(u)
        self.model._vars=self.model.getVars()
        self.model.params.LazyConstraints = 1
        self.model.params.TimeLimit = 3660
        if self.f>=self.height:
            self.model.optimize(add_cutting_plane)
        else:
            print("実行不可")

    def get_optimal_val(self):
        if self.model.Status==GRB.OPTIMAL:
            self.LB=self.model.ObjVal
            return self.model.ObjVal
        else:
            return None

    def get_optimal_sol(self):
        if self.model.Status==GRB.OPTIMAL:
            x_opt={}
            c_opt={}
            for var in self.model._vars:
                if "x_" in var.VarName:
                    # print(var)
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    p=int(var.VarName.split("_")[-1])
                    x_opt[s,h,p]=var.X
                if "c_" in var.VarName:
                    s=int(var.VarName.split("_")[-3])
                    h=int(var.VarName.split("_")[-2])
                    i=int(var.VarName.split("_")[-1])
                    c_opt[s,h,i]=var.X
                if "alpha" in var.VarName:
                    alpha_opt=var.X
                if "u" in var.VarName:
                    u_opt=var.X
            return (x_opt,c_opt,alpha_opt,u_opt)
        else:
            return None

    def get_result(self,x_tmp):
        self.x_tmp=x_tmp
        EPS=1.e-6
        self.result=np.zeros((self.height,self.stack_num))
        for (s,h,p) in self.x_tmp:
            if self.x_tmp[s,h,p]>EPS:
                self.result[self.height-h][s-1]=int(p)
        self.result=self.result.astype(int)
        return self.result
    
    # def get_penalty(self,result):
    #     if self.model.Status==GRB.OPTIMAL:
    #         self.result=result
    #         self.penalty=[]
    #         x,c,alpha,u=self.get_optimal_sol()
    #         for k in self.O:
    #             OO=k
    #             a=0
    #             for j in range(self.stack_num):
    #                 for i in range(1,self.height):
    #                     for i_ in range(i+1,self.height+1):
    #                         if self.result[i-1][j]!=0 and self.result[i_-1][j]!=0:
    #                             if OO.index(self.result[i-1][j])>OO.index(self.result[i_-1][j]):
    #                                 # print(j+1,height-i+1,O)
    #                                 a+=1
    #                                 # print("penalty!")
    #                                     # print(i,i_,j+1,O)
    #                                 break
    #             self.penalty.append(a)
    #         self.zk=[i for i, x in enumerate(self.penalty) if x-alpha>0]
    #         for zk_ in self.zk:
    #             self.OC.append(self.O[zk_])
    #         return (self.zk,self.OC,self.penalty)


In [3]:
# 1m10.9s
size=1000
size_=10000
beta=0.75
inst=[4,4,[2,2,2,2,2,2]]
pcvar=[]
import numpy as np
# mean=np.arange(1,ship_num+1)
mean=np.arange(1,len(inst[2])+1)
covl=[2 for i in range(len(inst[2]))]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov=np.zeros((len(inst[2]),len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i==j:
            cov[i][i]=covl[1]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i!=j:
            cov[i][j]=0

OC=[]
EPS=0.1
a=0
UB=1000
LB=0
UB_k=[]
zk=[]
aaa=[]

model=CuttingPlaneAlgorithm(len(inst[2]),inst[0],inst[1],inst[2],size,size_,0.75,mean,cov,OC,zk,aaa)
model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-30
OC= []
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 3660
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 108 rows, 13098 columns and 1856 nonzeros
Model fingerprint: 0xecf9713e
Variable types: 13002 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
[[0 0 0 2]
 [1 3 0 2]
 [3 5 1 4]
 [6 6 4 5]]
Len_dic= 152
770 770
Presolve removed 20 rows and 8 columns
Presolve time: 0.03s
Presolved: 88 rows, 13090 columns, 1620 nonzeros
Variable types: 13002 continuous, 88 integer (88 binary)
[[0 0 0 5]
 [1 1 0 5]
 [2 3 4 6]
 [2 3 4 6]]
Len_dic= 154
558 832

Root relaxation: objective 0.000000e+00, 435 iterations, 0.31 seconds (0.13 work u

In [4]:
# 1m22.8s
size=1000
size_=10000
beta=0.75
inst=[5,5,[2,2,2,2,2,2,2,2,2]]
pcvar=[]
import numpy as np
# mean=np.arange(1,ship_num+1)
mean=np.array([1,1.5,4,4.5,5,5.6,6,7.5,8])
covl=[1,1,1.5,1,1,1.5,1,1,1.5]
# a=list(a)
# a=random.sample(a,ship_num)
# mean=np.sort(a)
cov=np.zeros((len(inst[2]),len(inst[2])))
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i==j:
            cov[i][i]=covl[1]
for i in range(len(inst[2])):
    for j in range(len(inst[2])):
        if i!=j:
            cov[i][j]=0

OC=[]
EPS=0.1
a=0
UB=1000
LB=0
UB_k=[]
zk=[]
aaa=[]

model=BB(len(inst[2]),inst[0],inst[1],inst[2],size,size_,0.75,mean,cov,OC,zk,aaa)
model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-30
OC= []
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 208599 rows, 21227 columns and 2301162 nonzeros
Model fingerprint: 0x8045a3ec
Variable types: 21002 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [4e-03, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 5102 rows and 0 columns (presolve time = 5s) ...
Presolve removed 5102 rows and 0 columns (presolve time = 11s) ...
Presolve removed 21342 rows and 20 columns (presolve time = 15s) ...
Presolve removed 24902 rows and 20 columns (presolve time = 20s) ...
Presolve removed 24902 rows and 20 columns (presolve time = 26s) ...